# Final project
**Yuanxing Cheng, 810925466**
## Q1
$Code$

```MATLAB
function [ A ] = tridiag( S )
%UNTITLED Summary of this function goes here
%   Detailed explanation goes here

if ~issymmetric(S)
    disp('Not a symmetric matrix!');
    return
end

[~,n] = size(S);

if n < 3
    disp ('Matrix too small!');
    return
end

for k=1:n-2
    x = S(k+1:n,k);
    v = x;
    v(1) = x(1) + sign(x(1)) * norm(x);
    v = v/norm(v);
    S(k+1:n,k:n) = S(k+1:n,k:n) - 2*v*(v'*S(k+1:n,k:n));
    S(k:n,k+1:n) = S(k:n,k+1:n) - 2*(S(k:n,k+1:n)*v)*v';
end

%since there's some rounding errors so here I move the lower part to upper
%part

A = zeros(size(S));

for k=1:n
    for j=k:n
        A(k,j) = S(k,j);
        A(j,k) = S(k,j);
    end
end

end
```

and run this function with $S = \text{hilb}(4)$;

```MATLAB
>> S = hilb(4)
S =
   1.000000000000000   0.500000000000000   0.333333333333333   0.250000000000000
   0.500000000000000   0.333333333333333   0.250000000000000   0.200000000000000
   0.333333333333333   0.250000000000000   0.200000000000000   0.166666666666667
   0.250000000000000   0.200000000000000   0.166666666666667   0.142857142857143
```

```
>> tridiag(S)
ans =
   1.000000000000000  -0.650854139658888  -0.000000000000000  -0.000000000000000
  -0.650854139658888   0.650585480093677   0.063911879959868  -0.000000000000000
  -0.000000000000000   0.063911879959868   0.025320143416558  -0.001165208041306
  -0.000000000000000  -0.000000000000000  -0.001165208041306   0.000284852680241
```

## Q2
$Code$

```MATLAB
function [ Anew, ldiag1Thr] = qralg( A )
%UNTITLED2 Summary of this function goes here
%   Detailed explanation goes here

if ~issymmetric(A)
    disp('Not a symmetric matrix!');
    return
end

[m,~] = size(A);
k = 1;
ldiag1Thr=[];

while abs(A(m,m-1)) >= 10^(-12)
    [Q,R] = qr(A);
    A = R*Q;
    ldiag1Thr(k) = abs(A(m,m-1));
    k = k+1;
end

Anew = zeros(size(A));

for k=1:m
    for j=k:m
        Anew(k,j) = A(k,j);
        Anew(j,k) = A(k,j);
    end
end

ldiag1Thr = ldiag1Thr';

end
```

And then I run the function

```MATLAB
>> A = tridiag(hilb(4));
>> Anew = qralg(A)
Anew =
   1.500214279821008  -0.000017807525414  -0.000000000000000   0.000000000000000
  -0.000017807525414   0.169141220459685   0.000000005628744   0.000000000000000
  -0.000000000000000   0.000000005628744   0.006738273605761   0.000000000000689
   0.000000000000000   0.000000000000000   0.000000000000689   0.000096702304023
```

then I check that with the built-in function `eig`;

```MATLAB
>> eig(A)
ans =
   0.000096702304023
   0.006738273605761
   0.169141220221450
   1.500214280059243
```

It's obviously that the last entry in $Anew$ is exactly the smallest eigenvalue of $A$.

## Q3
I name the function in this question as `Thr`.

$Code$

```MATLAB
function [ eigV, ldiag1Thr ] = Thr( S )

if ~issymmetric(S)
    disp('Not a symmetric matrix!');
    return
end

[m,~] = size(S);

if m < 3
    disp ('Matrix too small!');
    return
end

eigV = zeros(m,1);
Anew = tridiag(S);
ldiag1Thr=[];

for k=m-1:-1:1
    A = Anew(1:k+1,1:k+1);
    [ Anew, ldiag1temp] = qralg( A );
    eigV(m-k) = Anew(k+1,k+1);
    ldiag1Thr = [ldiag1Thr;ldiag1temp];
end

eigV(m) = Anew(1);

p = semilogy(ldiag1Thr);
p.LineWidth = 1.5;
l = legend('\fontname{Courier New}\fontsize{20} \bf A(m,m-1)');

T = title('\fontname{Courier New}\fontsize{18} \bf Find the eigenvalues without shift');

Y = ylabel(['\fontname{Courier New} \bf Value of'; '\fontname{Courier New} \bf A(m,m-1)']);
X = xlabel('\fontname{Courier New} \bf Iteration');

set(Y, 'Units', 'Normalized', 'Position', [-0.1, 0.5, 0],'Rotation',0);
set(X, 'Units', 'Normalized', 'Position', [0.5, -0.07, 0]);

end
```

and the result is

```MATLAB
>> Thr(S)
ans =
   0.000096702304023
   0.006738273605761
   0.169141220221450
   1.500214280059243
```

Exactly the same!

$Figure$

![](./Fin/withoutShift.png)






## Q4
For future use I name the function in this question as `qralgFor`.

$Code$

```MATLAB
function [ Anew, ldiag1For] = qralgFor( A )
%UNTITLED2 Summary of this function goes here
%   Detailed explanation goes here

if ~issymmetric(A)
    disp('Not a symmetric matrix!');
    return
end

[m,~] = size(A);
k = 1;
I = eye(m);
ldiag1For=[];

while abs(A(m,m-1)) >= 10^(-12)
    delta = (A(m-1,m-1) - A(m,m))/2;
    mu = A(m,m) - sign(delta)*(A(m,m-1))^2 / ...
         (abs(delta) + sqrt(delta^2 + (A(m,m-1))^2));
    Im = mu*I;
    [Q,R] = qr(A-Im) ;
    A = R*Q + Im;
    ldiag1For(k) = abs(A(m,m-1));
    k = k+1;
end
```

```
Anew = zeros(size(A));

for k=1:m
    for j=k:m
        Anew(k,j) = A(k,j);
        Anew(j,k) = A(k,j);
    end
end

ldiag1For = ldiag1For';

end
```

And to redo the question 3, I just made a new similar function called `For`.

$Code$

```MATLAB
function [ eigV, ldiag1For ] = For( S )

if ~issymmetric(S)
    disp('Not a symmetric matrix!');
    return
end

[m,~] = size(S);

if m < 3
    disp ('Matrix too small!');
    return
end

eigV = zeros(m,1);
Anew = tridiag(S);
ldiag1For=[];

for k=m-1:-1:1
    A = Anew(1:k+1,1:k+1);
    [ Anew, ldiag1temp] = qralgFor( A );
    eigV(m-k) = Anew(k+1,k+1);
    ldiag1For = [ldiag1For;ldiag1temp];
end

eigV(m) = Anew(1);

p = semilogy(ldiag1For);
p.Color = 'black';
p.LineWidth = 1.5;
l = legend('\fontname{Courier New}\fontsize{20} \bf A(m,m-1)');
l.Location = 'southwest';

T = title('\fontname{Courier New}\fontsize{18} \bf Find the eigenvalues with Wilkinson shift');

Y = ylabel(['\fontname{Courier New} \bf Value of'; '\fontname{Courier New} \bf A(m,m-1)']);
X = xlabel('\fontname{Courier New} \bf Iteration');

set(Y, 'Units', 'Normalized', 'Position', [-0.1, 0.5, 0],'Rotation',0);
set(X, 'Units', 'Normalized', 'Position', [0.5, -0.07, 0]);

end
```

The result is 

```MATLAB
>> For(S)
ans =
   0.000096702304023
   0.006738273605761
   0.169141220221450
   1.500214280059242
```

$Figure$

![](./Fin/withWilkinsonShift.png)

## Q5
Here is the result in MATLAB console.

```MATLAB
>> S = diag(15 : -1 : 1)+ones(15, 15);
>> Thr(S)
ans =
   1.214655366493388
   2.256950983586658
   3.287775587950228
   4.314311852797305
   5.338959876554918
   6.362944488528983
   7.387092745473666
   8.412112071300873
   9.438745755142197
  10.467921659050409
  11.500983018469308
  12.540186373729330
  13.590131956252899
  14.664096999367858
  24.223131265302101
```

$Figure$

![](./Fin/withoutShiftMatrix.png)

```
>> For(S)
ans =
   6.362944488528973
   7.387092745473670
   5.338959876554913
   8.412112071300848
   9.438745755142206
  10.467921659050401
   4.314311852797315
  11.500983018469292
   3.287775587950219
  12.540186373729304
   2.256950983586659
  13.590131956252860
   1.214655366493377
  14.664096999367892
  24.223131265302101
```

$Figure$

![](./Fin/withWilkinsonShiftMatrix.png)